In [231]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
import csv

In [233]:
data = pd.DataFrame(columns=['name', 'image', 'description', 'location', 'speaking', 'rating', 'num_reviews' ,
                                     'currency', 'salary', 'num_proj', 'num_buyers', 'last_active', 'expertise',
                                      'skills', 'gender' ])

In [234]:
URL_list = ['https://www.peopleperhour.com/freelancer/marketing-seo/gajura-constantin-seo-expert-and-experienced-guest-wwanwy',
            'https://www.peopleperhour.com/freelancer/writing-translation/logical_translation_localisation-translations-translation-localisation-mnyay']

In [235]:
def guess_gender():
    """
    determine sex of a worker with a list of reviews
    """
    gender = ""
    male = ["he", "his", "him", "himself"]
    female = ["she", "her", "hers", "herself"]
    
    for r in reviews :
        for word in male :
            if re.compile(r'\b({0})\b'.format(word), flags=re.IGNORECASE).search(r) != None:
                gender = "male"
                break
            break
        for word in female : 
            if re.compile(r'\b({0})\b'.format(word), flags=re.IGNORECASE).search(r) != None:
                gender = "female"
                break
            break
    return gender

In [236]:
for URL in URL_list:
    
    r = requests.get(URL)
    soup = BeautifulSoup(r.content)
    
    location = soup.find("div", {"class": "member-location"}).get_text().replace('\n', '')
    speaking = soup.find("div", {"class": "member-languages clearfix"}).get_text().replace('\n', '')
    rating = soup.find("div", {"class": "total-rating"}).get_text()
    num_reviews = int(soup.find("div", {"class": "total-reviews"}).get_text().replace('(', '').replace(')', ''))
    cost = soup.find("span", {"class": "member-cost"}).get_text().replace('\n', '')
    currency = cost[0]
    salary = int(cost[1:].replace('/hr', ''))
    num_proj = int(re.sub("[^0-9]", "", soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['memStats-container']).find("div", {"class": "memberStats-item memberStats-rating"}).get_text().replace('\n', '')))
    num_buyers = int(re.sub("[^0-9]", "", soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['memStats-container']).find("div", {"class": "memberStats-item u-mgb--1"}).get_text().replace('\n', '')))
    last_active = soup.find("div", {"class": "memberStats-item item-margin"}).find("div", {"class": "insights-value"}).get_text().replace('\n', '')
    expertise = soup.find("div", {"class": "clearfix industry-expertise-list skills-item"}).get_text().replace('\n', '')
    response_time = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['memberStats-item']).find("div", {"class": "insights-value"}).get_text().replace('\n', '')
    skills = soup.find("div", {"class": "col-md-12 col-lg-12"}).get_text().replace('\n', '')
    
    profile = json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))
    name = profile['name']
    image = profile['image']
    description = profile['description']
    
    reviews = [com.find("p").get_text() for com in soup.findAll("li", {"class": "item participant feedback clearfix"})]
    gender = guess_gender()
    
    #worker = dict(((k, eval(k)) for k in ('name', 'image', 'description', 'location', 'speaking', 'rating', 'num_reviews' , 'currency', 'salary', 'num_proj', 'num_buyers', 'last_active', 'expertise', 'skills', 'gender' )))
    
    
    worker = [name, image, description, location, speaking, rating, num_reviews,
                                     currency, salary, num_proj, num_buyers, last_active, expertise,
                                      skills, gender]

    a_series = pd.Series(worker, index = data.columns)
    data = data.append(a_series, ignore_index=True)


In [237]:
data

,name,image,description,location,speaking,rating,num_reviews,currency,salary,num_proj,num_buyers,last_active,expertise,skills,gender
0,GAJURA Constantin,https://dw3i9sxi97owk.cloudfront.net/uploads/7...,"Hey, I'm Gajura Constantin! I've had a passion...","Berlin, Germany","German, English and Spanish speaking",5.0,1609,€,30,2207,896,Apr 20 2021,"Industry expertise: Advertising, Marke...",Skillsonline marketinglink buildinggoogle rank...,male
1,Logical Translation & Localisation,https://dw3i9sxi97owk.cloudfront.net/uploads/3...,OFFERING A WIDE RANGE OF TRANSLATION SERVICES\...,"City of London, United Kingdom","English, Spanish, French and Italian speaking",4.9,135,£,20,140,100,Apr 20 2021,"Industry expertise: Advertising, Aeros...",SkillseditingproofreadingGerman translationEng...,female
